In [53]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time

In [141]:
data = pd.read_excel('book2.xlsx', index_col=0)  
data = data.reset_index()
data = data.rename(columns={"index": "team"})

In [142]:
data


,team,3PM,3PA,3P%,3P% share in Total Points,Year
0,Atlanta Hawks,8,22.4,36,25.32,1996-97
1,Boston Celtics,5.7,16.2,35.1,16.99,1996-97
2,Brooklyn Nets,5.9,16.7,35.3,18.21,1996-97
3,Chicago Bulls,6.4,17.1,37.3,18.63,1996-97
4,Cleveland Cavaliers,5.9,15.7,37.6,20.23,1996-97
...,...,...,...,...,...,...
715,Toronto Raptors,13.8,37,37.4,36.7,2019-20
716,Utah Jazz,13.4,35.2,38,36.1,2019-20
717,Washington Wizards,12,32.6,36.8,31.5,2019-20
718,Charolette Hornets,13.1,38.1,34.3,35.2,2019-20
